<a href="https://colab.research.google.com/github/NandiniBhattad13/BYOP/blob/main/BYOPgradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio
!pip install datasets

In [2]:
import gradio as gr
import re

from transformers import pipeline

In [3]:
import datasets
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import tensorflow as tf
from tqdm import tqdm
from transformers import pipeline
from transformers import create_optimizer
from transformers import AutoTokenizer
from transformers import BertTokenizer
from transformers import BertModel
from datasets import load_dataset
from transformers import TFAutoModelForSequenceClassification
from transformers.keras_callbacks import KerasMetricCallback
from transformers import DataCollatorWithPadding

In [4]:

def sentiment(opn,text):

 if(opn == "Emotion"):
    split_name = "test"
    classifier = pipeline(task="text-classification", model="SamLowe/roberta-base-go_emotions", top_k=None)
    model_output = classifier(text)
    return model_output[0][0]['label']

 elif(opn == "AI or not"):
   classifier = pipeline(task="text-classification", model="Lau123/detect_ai_generated_text", top_k=None)
   model_output = classifier(text)
   s = model_output[0][0]['score']
   if(s>0.6):
     ans="AI"
   else:
     ans="HUMAN"
   return(ans)

 elif(opn == "POS or NEG"):

    imdb = load_dataset("imdb")
    tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
    def preprocess_function(examples):
      return tokenizer(examples["text"], truncation=True)
    tokenized_imdb = imdb.map(preprocess_function, batched=True)
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")
    id2label = {0: "NEGATIVE", 1: "POSITIVE"}
    label2id = {"NEGATIVE": 0, "POSITIVE": 1}
    from transformers import create_optimizer
    batch_size = 16
    num_epochs = 5
    batches_per_epoch = len(tokenized_imdb["train"]) // batch_size
    total_train_steps = int(batches_per_epoch * num_epochs)
    optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)
    model = TFAutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id)
    tf_train_set = model.prepare_tf_dataset(
    tokenized_imdb["train"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,)

    tf_validation_set = model.prepare_tf_dataset(
    tokenized_imdb["test"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,)

    model.compile(optimizer=optimizer)
    model.save_pretrained("saved_model")
    tokenizer.save_pretrained("saved_model")
    tokenizer = AutoTokenizer.from_pretrained("saved_model")
    inputs = tokenizer(text, return_tensors="tf")
    model = TFAutoModelForSequenceClassification.from_pretrained("saved_model")
    logits = model(**inputs).logits
    predicted_class_id = int(tf.math.argmax(logits, axis=-1)[0])
    predicted_class_id
    return(id2label[predicted_class_id])

 else:

   source_url = "/content/bbc-text.csv"
   df = pd.read_csv(source_url)
   df.groupby(["category"]).size()
   tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
   tokenizer= BertTokenizer.from_pretrained("bert-base-cased")
   labels = {
      'business':0,
      'entertainment':1,
      'sport':2,
      'tech':3,
      'politics':4
     }
   class Dataset(torch.utils.data.Dataset):
      def __init__(self,df):
          #extract our labels from the df
          self.labels = [labels[label] for label in df["category"]]
          #tokenize our texts to the format that BERT expects to get as input
          self.texts = [tokenizer(text, padding='max_length', max_length=512, truncation=True,return_tensors="pt") for text in df["text"]]
      def classes(self):
          return self.labels

      def __len__(self):
          return len(self.labels)

      #fetch a batch of labels
      def get_batch_labels(self,indx):
          return np.array(self.labels[indx])
      # fetch a batch of texts
      def get_batch_texts(self,indx):
          return self.texts[indx]

    #get an item with the texts and the label
      def __getitem__(self,indx):
        batch_texts = self.get_batch_texts(indx)
        batch_y = self.get_batch_labels(indx)
        return batch_texts, batch_y

   df_train, df_valid,df_test = np.split(df.sample(frac=1,random_state=42),[int(.8*len(df)), int(.9*len(df))])

   class BertClassifier(torch.nn.Module):
    def __init__(self,dropout=0.5):
        super(BertClassifier,self).__init__()

        self.bert=BertModel.from_pretrained("bert-base-cased")
        self.dropout = torch.nn.Dropout(dropout)
        # bert output a vector of size 768
        self.lin = torch.nn.Linear(768,5)
        self.relu = torch.nn.ReLU()
    def forward(self,input_id,mask):
        # as output, the bert model give us first the embedding vector of all the tokens of the sequence
        # second we get the embedding vector of the CLS token.
        # fot a classification task it's enough to use this embedding for our classifier
        _,pooled_output = self.bert(input_ids= input_id,attention_mask = mask,return_dict = False)
        dropout_output = self.dropout(pooled_output)
        linear_output  = self.lin(dropout_output)
        final_layer = self.relu(linear_output)
        return final_layer

   def train(model, train_data, val_data, learning_rate, epochs=5):
    #creating a custom Dataset objects using the training and validation data
    train, val = Dataset(train_data), Dataset(val_data)
    #creating dataloaders
    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr= learning_rate)

    if use_cuda:
            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader):
                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)
                output = model(input_id, mask)
                batch_loss = criterion(output, train_label)
                total_loss_train += batch_loss.item()

                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc
                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            total_loss_val = 0

   EPOCHS = 1
   model = BertClassifier()
   learning_rate = 1e-6
   train(model, df_train, df_valid, learning_rate, EPOCHS)
   device= torch.device("cuda" if torch.cuda.is_available() else "cpu")
   inverse_labels = {v:k for k,v in labels.items()}
   def predict(device,model,sentence):
    sentence = sentence
    sentence_input = tokenizer(sentence, padding='max_length', max_length=512, truncation=True,return_tensors="pt").to(device)
    input_id = sentence_input["input_ids"]
    mask = sentence_input["attention_mask"]
    output = model(input_id,mask)
    predicted_class_label = output.argmax(dim=1)
    predicted_class = inverse_labels[predicted_class_label.item()]
    print(predicted_class)
   predict(device,model,text)

In [5]:
demo = gr.Interface(
    fn=sentiment,
    inputs=[
        gr.Dropdown(["AI or not", "POS or NEG", "Emotion", "category of text"], label="Sentiment Analysis", info="What type of sentiment do you want to analyze?"),
        gr.Textbox(label="Enter Text for Sentiment Analysis")
    ],
    outputs="text"
)

In [6]:
demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://73a696a953cb486eba.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
